## Cargar datos de salarios DGII de Ene y Feb 2016

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from __future__ import division
from __future__ import print_function
from unidecode import unidecode
import re
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.style
#print(matplotlib.style.available)
#matplotlib.style.use('seaborn-colorblind')

df = pd.read_csv('cs_DGII_Nomina_2016.csv',sep=';')
print(df.shape)
print(df.columns)
df.head()

IOError: File cs_DGII_Nomina_2016.csv does not exist

Tenemos 5 columnas de datos. Incluyendo el género

En adición a cargar los datos realizamos una limpieza minima de los mismos para preparlos para el analisis. Esta consiste en:

* Convertir salarios de texto a montos
* Homogeneizar los puestos eliminando tildes, porciones entre parentesis, espacios en blanco de más y convirtiendo todas las letras a minusculas

In [ ]:
def normalize_puesto(pstr):
    pstr = unidecode(unicode(pstr,'windows-1252')).strip().lower()
    pstr = re.sub('\(.*?\)','', pstr)
    return pstr


df.columns = [x.strip().translate(None,',').lower() for x in df.columns.values]
# parse salaries
df.salario = df.salario.apply(lambda x: x.translate(None,',').strip())
df.salario = df.salario.astype(float)
# cleanup puesto
df['puesto_clean'] = df.puesto.apply(normalize_puesto)
df.drop(['puesto'],axis=1,inplace=True)
df.rename(columns={'puesto_clean':'puesto'},inplace=True)
print(df.shape)

In [ ]:
df.salario.describe()

In [ ]:
df['salariok'] = df.salario.div(1000).round(4)
df.salariok.describe()

In [ ]:
df.groupby(by='mes').salario.describe().unstack()

In [ ]:
ene = df[df.mes == 'ene-16']
feb = df[df.mes == 'feb-16']
print(ene.shape)
print(feb.shape)
print(ene.shape[0] + feb.shape[0])

In [ ]:
feb.salario.describe()

In [ ]:
print(feb.genero.value_counts())

In [ ]:
feb_by_gender = feb.groupby(by='genero')
feb_by_gender.salario.describe().unstack()

In [ ]:
print("\nProporción de empleomanía por genero")
print(feb_by_gender.salario.count().apply(lambda x: x/feb.salario.count()))
print("\nTotal nomina mensual por genero")
print(feb_by_gender.salario.sum())
print("\nProporcion de la nomina mensual por genero")
print(feb_by_gender.salario.sum().apply(lambda x: x/feb.salario.sum()))
print("\nDiferencia porcentual entre el salario promedio de mujeres y hombres")
print((feb_by_gender.salario.mean()[0]/feb_by_gender.salario.mean()[1] - 1)*100)

In [ ]:
print("Distribución de salarios por genero")
g = sns.FacetGrid(data=feb, col='genero', size=6).map(sns.distplot,'salariok')

In [ ]:
ene.puesto.nunique()

In [ ]:
feb.puesto.nunique()

In [ ]:
print(feb.puesto.value_counts().head(1))
print(ene.puesto.value_counts().head(1))

In [ ]:
set(feb.puesto.unique()) - set(ene.puesto.unique())

In [ ]:
df[df.puesto == 'disenador grafico']

Como lo que nos interesa analizar es el salario que devengan hombres y mujeres por el mismo trabajo debemos procurar comparar solo aquellos que tienen el mismo puesto.

Asi que tomamos los registros pertenecientes a puestos con 30 personas o mas y al menos 1 hombre y 1 mujer.

In [ ]:
spg = feb.groupby(by=['puesto']).filter(lambda x: len(x) > 14 and len(x[x.genero == 'F']) > 0 and len(x[x.genero == 'M']) > 0)
spg.puesto.value_counts()

In [ ]:
print(spg.puesto.nunique())
print(spg.puesto.value_counts().sum())
spg.groupby(by='genero').salario.describe().unstack()

In [ ]:
pd.pivot_table(data=spg,values='salario',index=['puesto'],columns=['genero'],aggfunc=np.mean)

In [ ]:
pivot_pg = pd.pivot_table(data=spg,values='salario',index=['puesto'],columns=['genero'],aggfunc=np.mean).apply(lambda x: x/x[1] - 1, axis=1)
pivot_pg.sort_values('F',inplace=True)

In [ ]:
pivot_pg_plot_data = pivot_pg.copy()
pivot_pg_plot_data['positive'] = pivot_pg_plot_data.F.map(lambda x: x>0)
pivot_pg_plot_data.sort_values('F',inplace=True)
f = plt.figure(figsize=(12,12))
ax = sns.barplot(data=pivot_pg_plot_data,y=pivot_pg.index,x='F',hue='positive',orient='h')
ax.set_xlabel('% diferencia promedio')
#ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.legend().remove()

In [ ]:
print(pivot_pg.mean())
print(pivot_pg_plot_data.positive.value_counts())

In [ ]:
pivot_pg.idxmin()

In [ ]:
spg[spg.puesto == 'encargado seccion c'].groupby(by='genero').salario.describe().unstack()

In [ ]:
pivot_pg.idxmax()

In [ ]:
spg[spg.puesto == 'auditor interno c'].groupby(by='genero').salario.describe().unstack()

In [ ]:
bottom25 = feb[feb.salario <= feb.salario.quantile(.25)]
top25 = feb[feb.salario >= feb.salario.quantile(.75)]
bottom25_by_gender = bottom25.groupby(by='genero')
bottom25_by_gender.salario.describe().unstack()

In [ ]:
top25_by_gender = top25.groupby(by='genero')
top25_by_gender.salario.describe().unstack()

In [ ]:
sns.FacetGrid(data=bottom25, col='genero').map(sns.distplot,'salariok')

In [ ]:
sns.FacetGrid(data=top25, col='genero').map(sns.distplot,'salariok')

In [ ]:
print(bottom25_by_gender.salario.count())
print(bottom25_by_gender.salario.count().apply(lambda x: x/bottom25.salario.count()))
print(bottom25_by_gender.salario.sum())
print(bottom25_by_gender.salario.sum().apply(lambda x: x/bottom25.salario.sum()))

In [ ]:
print(top25_by_gender.salario.count())
print(top25_by_gender.salario.count().apply(lambda x: x/top25.salario.count()))
print(top25_by_gender.salario.sum())
print(top25_by_gender.salario.sum().apply(lambda x: x/top25.salario.sum()))

In [ ]:
sns.set_style("white")
sns.set_context({"figure.figsize": (12, 10)})
# generar lista del total de salarios pagados por puesto
spg_salario_total = spg.groupby(by='puesto').salario.sum()
# generar lista de salario pagado a mujeres y hombres por puesto
spg_salario_f = spg[spg.genero == 'F'].groupby(by='puesto').salario.sum()
spg_salario_m = spg[spg.genero == 'M'].groupby(by='puesto').salario.sum()
# dibujar el total de salario por puesto en azul
salario_m_plot = sns.barplot(x=spg_salario_total.index, y=spg_salario_total, color ='blue')
# superponer el total de salario de mujeres por puesto en rojo
salario_f_plot = sns.barplot(x=spg_salario_total.index, y=spg_salario_f, color='red')
# girar los nombres de los puestos 90 grados para que esten verticales y sean legibles
for i in salario_f_plot.get_xticklabels():
    i.set_rotation(90)
salario_f_plot.set_ylabel('salario total pagado')
#
topbar = plt.Rectangle((0,0),1,1,fc="red", edgecolor = 'none')
bottombar = plt.Rectangle((0,0),1,1,fc='blue',  edgecolor = 'none')
l = plt.legend([bottombar, topbar], ['Hombres', 'Mujeres'], loc=1, ncol = 2, prop={'size':16})
l.draw_frame(False)

In [ ]:
sns.set_style("white")
sns.set_context({"figure.figsize": (12, 10)})
# generar lista del total de empleados por puesto
spg_puesto_total = spg.groupby(by='puesto').salario.count()
# generar lista empleados mujeres y hombres por puesto
spg_puesto_f = spg[spg.genero == 'F'].groupby(by='puesto').salario.count()
spg_puesto_m = spg[spg.genero == 'M'].groupby(by='puesto').salario.count()
# dibujar el total de empleados por puesto en azul
salario_m_plot = sns.barplot(x=spg_puesto_total.index, y=spg_puesto_total, color ='blue')
# superponer el total de empleadas (mujeres) por puesto en rojo
salario_f_plot = sns.barplot(x=spg_puesto_total.index, y=spg_puesto_f, color='red')
# girar los nombres de los puestos 90 grados para que esten verticales y sean legibles
for i in salario_f_plot.get_xticklabels():
    i.set_rotation(90)
salario_f_plot.set_ylabel('cantidad de empleados')
#
topbar = plt.Rectangle((0,0),1,1,fc="red", edgecolor = 'none')
bottombar = plt.Rectangle((0,0),1,1,fc='blue',  edgecolor = 'none')
l = plt.legend([bottombar, topbar], ['Hombres', 'Mujeres'], loc=1, ncol = 2)
l.draw_frame(False)

In [ ]:
ax = sns.factorplot(data=spg,x='puesto',y='salario',hue='genero',kind='violin', size=15)
ax.set_xticklabels(rotation=90)

In [ ]:
pd.pivot_table(data=top25,index=['puesto'],columns=['genero'],values='salario',aggfunc=np.mean).dropna().apply(lambda x: x/x[1] - 1,axis=1).mean()

In [ ]:
pd.pivot_table(data=bottom25,index=['puesto'],columns=['genero'],values='salario',aggfunc=np.mean).dropna().apply(lambda x: x/x[1] - 1,axis=1).mean()

In [ ]:
bins = range(5000,150000,2500)
ax = sns.distplot(feb.salario, kde=False, bins=bins)
sns.distplot(feb.salario[feb.genero == 'F'], kde=False, color='red', bins=bins)
print(feb.salario.quantile(.25))
print(feb.salario.quantile(.5))
print(feb.salario.quantile(.75))
plt.axvline(25000,linestyle='dashed')
plt.axvline(40000,linestyle='dashed')
plt.axvline(55000,linestyle='dashed')

In [ ]:
ax = sns.countplot(x='genero',data=feb[feb.salario > feb.salario.quantile(.75)])
plt.figure()
ax = sns.countplot(x='genero',data=feb[feb.salario < feb.salario.quantile(.25)])

In [ ]:
import scipy.stats as stats
def mwu(f,m):
    try:
        u, p = stats.mannwhitneyu(f,m)
    except:
        u, p = (float('NaN'),float('Nan'))
    return p*2
        
spg.groupby(by='puesto').apply(lambda x: mwu(x[x.genero=='F'].salario,x[x.genero=='M'].salario)).sort_values()

In [ ]:
spg[spg.puesto == 'analista'].groupby(by='genero').salario.describe().unstack()

In [ ]:
sns.factorplot(data=spg[spg.puesto=='analista'],x='puesto',y='salario',hue='genero',kind='box', size=5)

In [ ]:
rspg = spg.copy()
rspg.genero = np.random.permutation(rspg.genero)
rspg[rspg.puesto == 'analista'].groupby(by='genero').salario.describe().unstack()

In [ ]:
rpivot_pg = pd.pivot_table(data=rspg,values='salario',index=['puesto'],columns=['genero'],aggfunc=np.mean).apply(lambda x: x/x[1] - 1, axis=1)
rpivot_pg.sort_values('F',inplace=True)

In [ ]:
rpivot_pg_plot_data = rpivot_pg.copy()
rpivot_pg_plot_data['positive'] = rpivot_pg_plot_data.F.map(lambda x: x>0)
rpivot_pg_plot_data.sort_values('F',inplace=True)
f = plt.figure(figsize=(12,12))
ax = sns.barplot(data=rpivot_pg_plot_data,y=rpivot_pg.index,x='F',hue='positive',orient='h')
ax.set_xlabel('% diferencia promedio')
#ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.legend().remove()

In [ ]:
bins = range(5000,150000,2500)
feb['salario_bin'] = pd.qcut(feb.salario,10,precision=0)
feb.salario_bin.value_counts()
epg_count = pd.pivot_table(data=feb,index=['salario_bin'],columns=['genero'],values='salario',aggfunc=len).apply(lambda x: x/x.sum() * 100,axis=1)
epg_sum = pd.pivot_table(data=feb,index=['salario_bin'],columns=['genero'],values='salario',aggfunc=np.sum).apply(lambda x: x/x.sum() * 100,axis=1)

In [ ]:
ax = epg_count.plot(kind='area',stacked=True, grid=True)
ax.set_xlabel('Deciles de Salario')
ax.set_ylabel('Porcentaje de empleados')
ticks = ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.axhline(50,linestyle='dashed')

In [ ]:
ax = epg_sum.plot(kind='area',stacked=True)
ax.set_xlabel('Deciles de Salario')
ax.set_ylabel('Porcentaje de salario pagado')
ticks = ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.axhline(50,linestyle='dashed')